在过去的几年里，出于对深度学习强烈的兴趣， 许多公司、学者和业余爱好者开发了各种成熟的开源框架。 这些框架可以自动化基于梯度的学习算法中重复性的工作。 在[3.2](http://zh.d2l.ai/chapter_linear-networks/linear-regression-scratch.html#sec-linear-scratch)节中，我们只运用了： 
    1. 通过张量来进行数据存储和线性代数；
    2. 通过自动微分来计算梯度。 实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用， 现代深度学习库也为我们实现了这些组件。

本节将介绍如何通过使用深度学习框架来简洁地实现[3.2节](http://zh.d2l.ai/chapter_linear-networks/linear-regression-scratch.html#sec-linear-scratch)中的线性回归模型。

## 生成数据集

与[3.2节](http://zh.d2l.ai/chapter_linear-networks/linear-regression-scratch.html#sec-linear-scratch)中类似，我们首先生成数据集。

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

我们可以调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

使用data_iter的方式与我们在[3.2节](http://zh.d2l.ai/chapter_linear-networks/linear-regression-scratch.html#sec-linear-scratch)中使用data_iter函数的方式相同。为了验证是否正常工作，让我们读取并打印第一个小批量样本。 与[3.2节](http://zh.d2l.ai/chapter_linear-networks/linear-regression-scratch.html#sec-linear-scratch)不同，这里我们使用iter构造Python迭代器，并使用next从迭代器中获取第一项。

In [3]:
next(iter(data_iter))

[tensor([[-0.1678,  1.7272],
         [-0.2178, -0.8052],
         [-0.6036, -0.1643],
         [ 1.2687,  0.2449],
         [-0.5995, -0.1154],
         [ 1.6624,  0.1052],
         [ 1.1559, -0.7786],
         [-0.5059,  0.4686],
         [ 0.4880,  0.4750],
         [ 0.6378,  0.0958]]),
 tensor([[-2.0032],
         [ 6.4917],
         [ 3.5588],
         [ 5.9079],
         [ 3.3900],
         [ 7.1586],
         [ 9.1581],
         [ 1.5986],
         [ 3.5484],
         [ 5.1495]])]

## 定义模型

当我们在[3.2节](http://zh.d2l.ai/chapter_linear-networks/linear-regression-scratch.html#sec-linear-scratch)中实现线性回归时， 我们明确定义了模型参数变量，并编写了计算的代码，这样通过基本的线性代数运算得到输出。 但是，如果模型变得更加复杂，且当我们几乎每天都需要实现模型时，自然会想简化这个过程。 这种情况类似于为自己的博客从零开始编写网页。 做一两次是有益的，但如果每个新博客就需要工程师花一个月的时间重新开始编写网页，那并不高效。

对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。

回顾[图3.1.2](http://zh.d2l.ai/chapter_linear-networks/linear-regression.html#fig-single-neuron)中的单层网络架构， 这一单层被称为全连接层（fully-connected layer）， 因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。

在PyTorch中，全连接层在Linear类中定义。 值得注意的是，我们将两个参数传递到nn.Linear中。 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [4]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## 初始化模型参数

在使用net之前，我们需要初始化模型参数。 如在线性回归模型中的权重和偏置。 深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

正如我们在构造nn.Linear时指定输入和输出尺寸一样， 现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数

计算均方误差使用的是MSELoss类，也称为平方$L_2$范数。 默认情况下，它返回所有样本损失的平均值。

In [6]:
loss = nn.MSELoss()

## 定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

通过深度学习框架的高级API来实现我们的模型只需要相对较少的代码。 我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。 当我们需要更复杂的模型时，高级API的优势将大大增加。 当我们有了所有的基本组件，训练过程代码与我们从零开始实现时所做的非常相似。

回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:

- 通过调用net(X)生成预测并计算损失l（前向传播）。
- 通过进行反向传播来计算梯度。
- 通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        # 前向传播，求预测并求损失
        l = loss(net(X), y)
        # 梯度清零
        trainer.zero_grad()
        # 反向传播，求梯度
        l.backward()
        # 更新模型参数
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000266
epoch 2, loss 0.000104
epoch 3, loss 0.000103


下面我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。 要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。 正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [10]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0006, 0.0005])
b的估计误差： tensor([0.0002])


## 小结

- 我们可以使用PyTorch的高级API更简洁地实现模型。
- 在PyTorch中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层和常见损失函数。
- 我们可以通过_结尾的方法将参数替换，从而初始化参数。

In [13]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

# 累积误差
loss = nn.MSELoss(reduction='sum')

trainer = torch.optim.SGD(net.parameters(), lr=0.03)

num_epochs = 3
for i in range(num_epochs):
    for X, y in data_iter:
        # 前向传播，求预测
        y_hat = net(X)
        # 计算损失
        l = loss(y_hat, y)
        # 梯度清零
        trainer.zero_grad()
        # 反向传播
        l.backward()
        # 更新参数
        trainer.step()
    train_y_hat = net(features)
    train_l = loss(train_y_hat, labels)
    print(f'epoch: {epoch + i}, loss: {train_l}')

w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

epoch: 2, loss: 0.00028365899925120175
epoch: 3, loss: 0.00010428927635075524
epoch: 4, loss: 0.00010341791494283825
w的估计误差： tensor([0.0004, 0.0003])
b的估计误差： tensor([0.0004])


## 练习

1. 如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？

将损失函数的定义方式中的reduction字段改为 'sum'的话，会使得梯度值放大为原来的num_example倍

使得在不改变学习率的情况下 很容易出现在最优解附近震荡的情况 降低学习效果

- 将损失函数的计算方式定义为整个损失的和
```python
loss = nn.MSELoss(reduction='sum')
trainer = torch.optim.SGD(net.parameters(), lr=0.03)  #指定需要优化的参数 还有学习率
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y) #这里的net返回输入x经过定义的网络所计算出的值
        trainer.zero_grad() #清除上一次的梯度值 避免叠加上下面的数据所带来的梯度
        l.backward() #损失函数进行反向传播 求参数的梯度 
        trainer.step()  #trainer步进 根据指定的优化算法进行参数的寻优
    l = loss(net(features), labels) #根据上面的参数优化结果计算参数对整个数据集的拟合状态 以loss进行反映
    print(f'epoch {epoch + 1}, loss {l:f}')
```

- 在上述情况下改变学习率
```python
trainer = torch.optim.SGD(net.parameters(), lr=0.03/10)  #指定需要优化的参数 还有学习率
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)  #这里的net返回输入x经过定义的网络所计算出的值
        trainer.zero_grad()  #清除上一次的梯度值 避免叠加上下面的数据所带来的梯度
        l.backward()  #损失函数进行反向传播 求参数的梯度
        trainer.step()  #trainer步进 根据指定的优化算法进行参数的寻优
    l = loss(net(features), labels)  #根据上面的参数优化结果计算参数对整个数据集的拟合状态 以loss进行反映
    print(f'epoch {epoch + 1}, loss {l:f}')
```

### 总结

明显可见地选用sum而不是mean将很大程度影响模型的学习效果

原因：若不将损失函数采用均值 将会使得参数的梯度过大

梯度经过放大后 原有的学习率显得过大 使得其出现了振荡 即步长过长导致

在最优解的附近震荡 而无法高效逼近最优点

- 以平均值作为损失函数
```text
epoch 1, loss 0.000104
epoch 2, loss 0.000104
epoch 3, loss 0.000104
```

- 以和作为损失函数，不改变学习率
```text
epoch 1, loss 0.108244
epoch 2, loss 0.123402
epoch 3, loss 0.105799
```

- 以和作为损失函数，改变学习率
```text
epoch 1, loss 0.096369
epoch 2, loss 0.096502
epoch 3, loss 0.095959
```

总结：默认情况下以mean作为损失的计算结果更好

2. 查看深度学习框架文档，它们提供了哪些损失函数和初始化方法？用Huber损失代替原损失，即

$$
l(y,y')=\begin{cases}
|y-y'|-\frac{\sigma}{2},\quad if|y-y'|\gt\sigma \\
\frac{1}{2\sigma}(y-y')^2, \quad 其他情况
\end{cases}
\tag{3.3.1}
$$

损失函数：
> torch.nn包

|                                函数                                |                                              作用                                              |
|:----------------------------------------------------------------:|:--------------------------------------------------------------------------------------------:|
|                    L1Loss(size_average=True)                     |                                          差的绝对值的平均值                                           |
|                    MSELoss(size_average=True)                    |                                             均方误差                                             |
|         CrossEntropyLoss(weight=None, size_average=True)         |                                  将LogSoftMax和NLLLoss集成到一个类                                   |
|             NLLLoss(weight=None, size_average=True)              |                                   负的log likelihood loss损失                                    |
|            NLLLoss2d(weight=None, size_average=True)             |                      对于图片的 negative log likehood loss。计算每个像素的 NLL loss                       |
|            KLDivLoss(weight=None, size_average=True)             |                                           KL 散度损失                                            |
|             BCELoss(weight=None, size_average=True)              |                                            二进制交叉熵                                            |
|          MarginRankingLoss(margin=0, size_average=True)          | 创建一个标准，给定输入 $x1$,$x2$两个1-D mini-batch Tensor's，和一个$y$(1-D mini-batch tensor) ,$y$里面的值只能是-1或1 |
|              HingeEmbeddingLoss(size_average=True)               |         给定一个输入 $x$(2-D mini-batch tensor)和对应的 标签 $y$ (1-D tensor,1,-1)，此函数用来计算之间的损失值         |
|             MultiLabelMarginLoss(size_average=True)              |                            计算多标签分类的 hinge loss(margin-based loss)                            |
|                 SmoothL1Loss(size_average=True)                  |                                          平滑版L1 loss                                          |
|                SoftMarginLoss(size_average=True)                 |                                 创建一个标准，用来优化2分类的logistic loss                                 |
|     MultiLabelSoftMarginLoss(weight=None, size_average=True)     |                    创建一个标准，基于输入x和目标y的 max-entropy，优化多标签 one-versus-all 的损失                    |
|         CosineEmbeddingLoss(margin=0, size_average=True)         |                                     使用cosine距离测量两个输入是否相似                                     |
| MultiMarginLoss(p=1, margin=1, weight=None, size_average=True)   |                  计算multi-class classification的hinge loss（magin-based loss）                   |

初始化方法：

| nonlinearity |             gain             |
|:------------:|:----------------------------:|
|    linear    |              	1              |
| conv{1,2,3}d |              	1              |
|   sigmoid    |              	1              |
|     tanh     |             	5/3             |
|    relu	   |           sqrt(2)            |
|  leaky_relu  | sqrt(2/(1+negative_slope^2)) |

3. 如何访问线性回归的梯度？

In [15]:
net[0].weight.grad, net[0].bias.grad

(tensor([[-0.0048,  0.0053]]), tensor([0.0050]))